--- Day 6: Wait For It ---
The ferry quickly brings you across Island Island. After asking around, you discover that there is indeed normally a large pile of sand somewhere near here, but you don't see anything besides lots of water and the small island where the ferry has docked.

As you try to figure out what to do next, you notice a poster on a wall near the ferry dock. "Boat races! Open to the public! Grand prize is an all-expenses-paid trip to Desert Island!" That must be where the sand comes from! Best of all, the boat races are starting in just a few minutes.

You manage to sign up as a competitor in the boat races just in time. The organizer explains that it's not really a traditional race - instead, you will get a fixed amount of time during which your boat has to travel as far as it can, and you win if your boat goes the farthest.

As part of signing up, you get a sheet of paper (your puzzle input) that lists the time allowed for each race and also the best distance ever recorded in that race. To guarantee you win the grand prize, you need to make sure you go farther in each race than the current record holder.

The organizer brings you over to the area where the boat races are held. The boats are much smaller than you expected - they're actually toy boats, each with a big button on top. Holding down the button charges the boat, and releasing the button allows the boat to move. Boats move faster if their button was held longer, but time spent holding the button counts against the total race time. You can only hold the button at the start of the race, and boats don't move until the button is released.

For example:

Time:      7  15   30
Distance:  9  40  200

This document describes three races:

The first race lasts 7 milliseconds. The record distance in this race is 9 millimeters.

The second race lasts 15 milliseconds. The record distance in this race is 40 millimeters.

The third race lasts 30 milliseconds. The record distance in this race is 200 millimeters.

Your toy boat has a starting speed of zero millimeters per millisecond. For each whole millisecond you spend at the beginning of the race holding down the button, the boat's speed increases by one millimeter per millisecond.

So, because the first race lasts 7 milliseconds, you only have a few options:

Don't hold the button at all (that is, hold it for 0 milliseconds) at the start of the race. The boat won't move; it will have traveled 0 millimeters by the end of the race.
Hold the button for 1 millisecond at the  start of the race. Then, the boat will travel at a speed of 1 millimeter per millisecond for 6 milliseconds, reaching a total distance traveled of 6 millimeters.
Hold the button for 2 milliseconds, giving the boat a speed of 2 millimeters per millisecond. It will then get 5 milliseconds to move, reaching a total distance of 10 millimeters.
Hold the button for 3 milliseconds. After its remaining 4 milliseconds of travel time, the boat will have gone 12 millimeters.
Hold the button for 4 milliseconds. After its remaining 3 milliseconds of travel time, the boat will have gone 12 millimeters.
Hold the button for 5 milliseconds, causing the boat to travel a total of 10 millimeters.
Hold the button for 6 milliseconds, causing the boat to travel a total of 6 millimeters.
Hold the button for 7 milliseconds. That's the entire duration of the race. You never let go of the button. The boat can't move until you let go of the button. Please make sure you let go of the button so the boat gets to move. 0 millimeters.
Since the current record for this race is 9 millimeters, there are actually 4 different ways you could win: you could hold the button for 2, 3, 4, or 5 milliseconds at the start of the race.

In the second race, you could hold the button for at least 4 milliseconds and at most 11 milliseconds and beat the record, a total of 8 different ways to win.

In the third race, you could hold the button for at least 11 milliseconds and no more than 19 milliseconds and still beat the record, a total of 9 ways you could win.

To see how much margin of error you have, determine the number of ways you can beat the record in each race; in this example, if you multiply these values together, you get 288 (4 * 8 * 9).

Determine the number of ways you could beat the record in each race. What do you get if you multiply these numbers together?

In [1]:
test_input = """
Time:      7  15   30
Distance:  9  40  200
"""

In [2]:
from dataclasses import dataclass
from __future__ import annotations
from math import ceil, floor, sqrt


@dataclass
class BoatRace:
    time: int
    record_distance: int


    def distance_from_time_held(self, time_held: int) -> int:
        velocity_mm_per_sec = time_held
        time_moving = max(0, self.time - time_held)
        return velocity_mm_per_sec * time_moving

    @property
    def times_exact_record(self) -> list[float]:
        # solve quadratic equation for zeroes
        # distance = - time_held **2 + race_time * time_held
        # diff_from_record =  - time_held **2 + race_time * time_held - record_distance
        # x = time_held, 
        a = -1
        b = self.time
        c = -self.record_distance
        zeros_list = sorted([
            (-b - sqrt(b**2 - 4 * a * c)) / (2 * a),
            (-b + sqrt(b**2 - 4 * a * c)) / (2 * a),
        ])
        return sorted(zeros_list)
    

    @property
    def count_winning_options(self) -> int:
        # using calculus and algebra (graph of a parabola)
        # we know that the winning values are those integers between the two zeros
        # limited to not be negative times or greater than total race time
        winning_range_start = max(0, 1+floor(self.times_exact_record[0]))
        winning_range_end_inclusive = min(self.time, ceil(self.times_exact_record[1])-1)
        return winning_range_end_inclusive - winning_range_start + 1




@dataclass
class BoatRaceGroup:
    boatrace_list: list[BoatRace]


    @classmethod
    def from_str(cls, input_str: str, ignore_spaces: bool = False) -> BoatRaceGroup:
        time_list = []
        record_distance_list = []
        for str_line in input_str.strip().split("\n"):
            line_type_str, line_values_str = str_line.split(":")
            if line_type_str == "Time":
                if ignore_spaces:
                    time_list = [int(line_values_str.replace(" ", ""))]
                else:
                    time_list = [int(x) for x in line_values_str.split()]
            if line_type_str == "Distance":
                if ignore_spaces:
                    record_distance_list = [int(line_values_str.replace(" ", ""))]
                else:
                    record_distance_list = [int(x) for x in line_values_str.split()]
        
        # assert that these are no longer empty lists
        assert time_list
        assert record_distance_list
        
        boat_race_list = [BoatRace(time=t, record_distance=rd) for (t, rd) in zip(time_list, record_distance_list)]
        return cls(boatrace_list=boat_race_list)


    @property
    def product_race_win_count(self) -> int:
        product_out = 1
        for b in self.boatrace_list:
            product_out *= b.count_winning_options
        return product_out


In [3]:
test_boatraces = BoatRaceGroup.from_str(test_input)

In [4]:

[x.count_winning_options for x in test_boatraces.boatrace_list]

[4, 8, 9]

In [5]:
[x.times_exact_record for x in test_boatraces.boatrace_list]

[[1.6972243622680054, 5.302775637731995],
 [3.4688711258507254, 11.531128874149275],
 [10.0, 20.0]]

In [6]:
test_boatraces.product_race_win_count

288

In [7]:
assert test_boatraces.product_race_win_count == 288

In [8]:
with open("../inputs/day6_input.txt") as f:
    input_str = f.read()

In [9]:
boatraces = BoatRaceGroup.from_str(input_str)

In [10]:
boatraces.product_race_win_count

6209190

# Part 2

--- Part Two ---
As the race is about to start, you realize the piece of paper with race times and record distances you got earlier actually just has very bad kerning. There's really only one race - ignore the spaces between the numbers on each line.

So, the example from before:

Time:      7  15   30
Distance:  9  40  200
...now instead means this:

Time:      71530
Distance:  940200
Now, you have to figure out how many ways there are to win this single race. In this example, the race lasts for 71530 milliseconds and the record distance you need to beat is 940200 millimeters. You could hold the button anywhere from 14 to 71516 milliseconds and beat the record, a total of 71503 ways!

How many ways can you beat the record in this one much longer race?

In [11]:
test_boatraces_2 = BoatRaceGroup.from_str(test_input, ignore_spaces=True)


In [12]:
# can still use the product of count which in this case will trivially take the product of a single race
assert test_boatraces_2.product_race_win_count == 71503

In [13]:
boatraces_2 = BoatRaceGroup.from_str(input_str, ignore_spaces=True)

boatraces_2.product_race_win_count

28545089